#Installation

#Importation

In [1]:
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import warnings
import random
from string import punctuation
import seaborn as sns

from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

# warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize


from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score

#Data Loading

In [4]:
from google.colab import drive
drive.mount('/content/drive')
fake_data=pd.read_csv("/content/drive/MyDrive/Fake.csv")
true_data=pd.read_csv("/content/drive/MyDrive/True.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#Data preparation

In [6]:
import numpy

In [7]:
true_data["label"]=numpy.ones(21417,dtype=int)
true_data

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [8]:
fake_data["label"]=numpy.zeros(23481,dtype=int)
fake_data

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
data=pd.concat([true_data,fake_data],ignore_index=True)
data

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
sw = stopwords.words('english')

new_words=('’','“', '”')

for i in new_words:
    sw.append(i)


# Convert to lower case
data['text'] = data['text'].str.lower()

# Tokenizing
data['tokenized_text'] = data['text'].apply(word_tokenize)

# Remove stopwords
data['filtered_text'] = data['tokenized_text'].apply(lambda x: [item for item in x if item not in sw])

# Remove punction
data['filtered_text'] = data['filtered_text'].apply(lambda x: [item for item in x if item not in punctuation])

# Check results
print(len(data['text'].iloc[0]),
      len(data['tokenized_text'].iloc[0]),
      len(data['filtered_text'].iloc[0]))

4659 883 455


In [25]:
vectorizer = TfidfVectorizer(stop_words=sw,lowercase=True)
y = data.label
x = vectorizer.fit_transform(data.text)

In [35]:
vectorizer.vocabulary_

{'washington': 116722,
 'reuters': 92410,
 'head': 50365,
 'conservative': 28093,
 'republican': 91850,
 'faction': 40762,
 'congress': 27954,
 'voted': 115912,
 'month': 73075,
 'huge': 52885,
 'expansion': 40209,
 'national': 75309,
 'debt': 31729,
 'pay': 82457,
 'tax': 106695,
 'cuts': 30454,
 'called': 22336,
 'fiscal': 42488,
 'sunday': 104808,
 'urged': 113884,
 'budget': 21235,
 'restraint': 92196,
 '2018': 4152,
 'keeping': 60533,
 'sharp': 98477,
 'pivot': 84319,
 'way': 116911,
 'among': 11772,
 'republicans': 91854,
 'representative': 91783,
 'mark': 68730,
 'meadows': 70125,
 'speaking': 102005,
 'cbs': 23669,
 'face': 40681,
 'nation': 75308,
 'drew': 35924,
 'hard': 49767,
 'line': 65456,
 'federal': 41612,
 'spending': 102131,
 'lawmakers': 64027,
 'bracing': 20119,
 'battle': 16499,
 'january': 57574,
 'return': 92378,
 'holidays': 52035,
 'wednesday': 117153,
 'begin': 17015,
 'trying': 110976,
 'pass': 82107,
 'fight': 42149,
 'likely': 65330,
 'linked': 65503,
 'iss

In [26]:
print (x.shape)
print (y.shape)

(44898, 121858)
(44898,)


#Logistic regression

In [50]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.25)


In [51]:
labels = NB_classifier.predict(X_test)

In [52]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
model.score(X_test, y_test)

0.9869042316258352